In [24]:
import openai, json

client = openai.OpenAI()

# ============================================================
# 1. Memory structures
# ============================================================
messages = []          # Full conversation history (LLM memory)
user_profile = {       # Structured memory for quick reference
    "favorite_genres": [],
    "watched_movies": [],
}

In [25]:
# ============================================================
# 2. System prompt — instructs the LLM HOW to use memory
# ============================================================
SYSTEM_PROMPT = """You are a Movie Expert Chatbot that remembers everything the user tells you.

YOUR MEMORY RULES:
1. When the user mentions genres they like, remember them for all future responses.
2. When the user mentions movies they've already watched, remember them and NEVER recommend those movies again.
3. When recommending movies, ALWAYS check your conversation history first:
   - Recommend movies matching the user's preferred genres.
   - Exclude any movie the user said they already watched.
   - Reference what you know about the user to show you remember.
4. If the user asks what you remember about them, accurately recall their favorite genres and watched movies from the conversation.

Be enthusiastic about movies."""

messages.append({"role": "system", "content": SYSTEM_PROMPT})


In [26]:
# ============================================================
# 3. Helper — update structured profile from user message
# ============================================================
def update_user_profile(user_msg: str):
    """
    Use a cheap LLM call to extract genres and watched movies
    from the user's message and update the profile.
    This runs silently — the user never sees it.
    """
    extraction_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Extract any favorite genres and already-watched movie titles "
                    "from the user message. Return ONLY valid JSON with two keys: "
                    '"genres" (list of strings) and "watched" (list of strings). '
                    "If nothing found, return empty lists. No markdown, no explanation."
                ),
            },
            {"role": "user", "content": user_msg},
        ],
    )

    try:
        extracted = json.loads(extraction_response.choices[0].message.content)
        for genre in extracted.get("genres", []):
            if genre not in user_profile["favorite_genres"]:
                user_profile["favorite_genres"].append(genre)
        for movie in extracted.get("watched", []):
            if movie not in user_profile["watched_movies"]:
                user_profile["watched_movies"].append(movie)
    except (json.JSONDecodeError, AttributeError):
        pass  # Extraction failed silently — main chat still works

In [27]:
# ============================================================
# 4. Build context-aware prompt with current profile state
# ============================================================
def build_profile_reminder():
    """Inject a concise profile summary so the LLM never 'forgets'."""
    if not user_profile["favorite_genres"] and not user_profile["watched_movies"]:
        return None

    parts = []
    if user_profile["favorite_genres"]:
        parts.append(f"Favorite genres: {', '.join(user_profile['favorite_genres'])}")
    if user_profile["watched_movies"]:
        parts.append(f"Already watched: {', '.join(user_profile['watched_movies'])}")

    return (
        "[MEMORY SNAPSHOT — use this to personalize your response]\n"
        + "\n".join(parts)
        + "\nDo NOT recommend any movie listed above as already watched."
    )



In [28]:
# ============================================================
# 5. Core chat function
# ============================================================
def chat(user_input: str):
    # A) Extract profile data from user message
    update_user_profile(user_input)

    # B) Append user message to conversation history
    messages.append({"role": "user", "content": user_input})

    # C) Build messages payload — inject profile reminder as a
    #    system message right before the LLM call so it's fresh
    call_messages = list(messages)  # shallow copy
    reminder = build_profile_reminder()
    if reminder:
        call_messages.append({"role": "system", "content": reminder})

    # D) Call LLM
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=call_messages,
    )

    ai_message = response.choices[0].message.content

    # E) Append assistant response to conversation history
    messages.append({"role": "assistant", "content": ai_message})

    return ai_message


In [ ]:
# ============================================================
# 6. Main loop (terminal) — swap this for Jupyter cells below
# ============================================================
if __name__ == "__main__":
    print("=== Movie Recommendation Chatbot (with Memory) ===")
    print('Type your message (or "q" to quit)\n')

    while True:
        user_input = input("User: ")
        if user_input.lower() in ("quit", "q"):
            break

        response = chat(user_input)
        print(f"AI: {response}\n")

    # Show final memory state
    print("\n--- User Profile (Memory) ---")
    print(json.dumps(user_profile, ensure_ascii=False, indent=2))

=== Movie Recommendation Chatbot (with Memory) ===
Type your message (or "q" to quit)

AI: Awesome choice! Science fiction is such a fantastic genre with endless possibilities! If you're ever looking for recommendations or want to chat about your favorite SF movies, I'm here for it! What else do you enjoy about the genre?

AI: Got it! You've already seen "Interstellar" and "Inception." Both are fantastic SF films! If you're in the mood for more sci-fi, let me know, and I'll find some great recommendations for you that you haven't seen yet!

AI: Absolutely! Here are some great science fiction movies you might enjoy tonight, excluding the ones you've already seen:

1. **Blade Runner 2049** - A visually stunning sequel that explores the themes of identity and humanity.
2. **The Martian** - A thrilling survival story that blends science and humor as an astronaut tries to survive on Mars.
3. **Ex Machina** - A thought-provoking film that delves into artificial intelligence and the nature of